In [1]:
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Dropout, BatchNormalization, Activation
from tensorflow.python.keras.optimizer_v2 import rmsprop
from sklearn.model_selection import train_test_split
import pandas as pd
import numpy as np
np.random.seed(1337)  # for reproducibility
from keras.preprocessing import sequence
from keras.utils import np_utils
from keras.models import Sequential
from keras.layers import Dense, Dropout, Activation, Embedding
from keras.layers import LSTM, SimpleRNN, GRU
from keras.utils.np_utils import to_categorical
from sklearn.metrics import (precision_score, recall_score,f1_score, accuracy_score,mean_squared_error,mean_absolute_error)
from sklearn import metrics
from sklearn.preprocessing import Normalizer
import h5py
from keras import callbacks
from keras.callbacks import ModelCheckpoint, EarlyStopping, ReduceLROnPlateau, CSVLogger
from keras.models import Sequential
from keras.layers import Dense
import tensorflow as tf
import keras
from keras.utils.np_utils import to_categorical
from keras import models
from keras import layers
import numpy
import sys
from sklearn.preprocessing import LabelEncoder, OneHotEncoder, MinMaxScaler
import math
from sklearn.externals import joblib
import joblib
from sklearn.preprocessing import StandardScaler
from keras.layers import BatchNormalization

Using TensorFlow backend.
E:\ProgramData\Anaconda3\envs\tf_gpu\lib\site-packages\sklearn\externals\joblib\__init__.py:15: FutureWarning: sklearn.externals.joblib is deprecated in 0.21 and will be removed in 0.23. Please import this functionality directly from joblib, which can be installed with: pip install joblib. If this warning is raised when loading pickled models, you may need to re-serialize those models with scikit-learn 0.21+.
  warnings.warn(msg, category=FutureWarning)


In [2]:
## required for efficient GPU use
import tensorflow as tf
from keras.backend import tensorflow_backend
config = tf.ConfigProto(gpu_options=tf.GPUOptions(allow_growth=True))
session = tf.Session(config=config)
tensorflow_backend.set_session(session)

In [3]:
#check if notebook is using GPU
from tensorflow.python.client import device_lib
print(device_lib.list_local_devices())

[name: "/device:CPU:0"
device_type: "CPU"
memory_limit: 268435456
locality {
}
incarnation: 15381463214726146654
, name: "/device:GPU:0"
device_type: "GPU"
memory_limit: 6624540754
locality {
  bus_id: 1
  links {
  }
}
incarnation: 1496851472889235876
physical_device_desc: "device: 0, name: GeForce RTX 2060 SUPER, pci bus id: 0000:01:00.0, compute capability: 7.5"
]


In [4]:
# load  preprocessed dataset
data=pd.read_csv("Train.csv")
#convert data from dataframe to array
data_new=data.values

#Split data into training data (X) and target label(Y)
x=data_new[:,:-1] #gets elements of array but the last one
y=data_new[:,-1]

#train-test split
from sklearn.model_selection import train_test_split
x_train, x_val, y_train, y_val = train_test_split( x,y, test_size=0.25)
print(x_train.shape)
print(x_val.shape)


(785336, 20)
(261779, 20)


In [5]:
def attack_detection_model(input_dim=20,dropout1_rate=0.2,dropout2_rate=0.25,dropout3_rate=0.3,dropout4_rate=0.3):

    # Define the model
    model = Sequential()
  
    model.add(Dense(1024,input_dim=input_dim,activation='relu',name="dense_1"))  
    model.add(Dropout(dropout1_rate,name="dropout_1"))
    model.add(Dense(1024,activation='relu',name="dense_2"))  
    model.add(Dropout(dropout2_rate, name="dropout_2"))
    model.add(Dense(512,activation='relu',name="dense_3"))  
    model.add(Dropout(dropout3_rate, name="dropout_3"))
    model.add(Dense(256,activation='relu',name="dense_4"))  
    model.add(Dropout(dropout4_rate, name="dropout_4"))
    model.add(Dense(1,name="dense_5"))
    model.add(Activation('sigmoid'))

    return model


In [6]:
def fit_with(input_dim, verbose,dropout1_rate, dropout2_rate,dropout3_rate,dropout4_rate,learning_rate):

    # Create the model using a specified hyperparameters.
    model = attack_detection_model(input_dim, dropout1_rate,dropout2_rate, dropout3_rate,dropout4_rate)

    # Train the model for a specified number of epochs.
    optimizer = keras.optimizers.Adam(lr=learning_rate)
    model.compile(loss='binary_crossentropy',
                  optimizer=optimizer,
                  metrics=['accuracy'])
    from time import time
    t0 = time()

    # Train the model with the train dataset.
    model.fit(x_train,y_train,validation_data=(x_val, y_val),epochs=100,shuffle=True,
              batch_size=32, verbose=verbose)
    tf = time() - t0
    print ("DNN model trained in {} seconds".format(round(tf, 3)))

    # Evaluate the model with the eval dataset.
    
    score = model.evaluate(x_val, y_val, batch_size=32,verbose=3)
    print('val loss:', score[0])
    print('val accuracy:', score[1])

    # Return the accuracy.

    return score[1]

from functools import partial
input_dim=20
verbose = 3
fit_with_partial = partial(fit_with, input_dim, verbose)

In [7]:
from bayes_opt import BayesianOptimization

# Bounded region of parameter space
pbounds = {'dropout1_rate': (0.1, 0.3),'dropout2_rate': (0.1, 0.3),'dropout3_rate': (0.1, 0.3),'dropout4_rate': (0.1, 0.3), 'learning_rate': (1e-3, 1e-2)}

optimizer = BayesianOptimization(
    f=fit_with_partial,
    pbounds=pbounds,
    verbose=3,  # verbose = 1 prints only when a maximum is observed, verbose = 0 is silent
    random_state=8)

optimizer.maximize(init_points=10, n_iter=10)


for i, res in enumerate(optimizer.res):
    print("Iteration {}: \n\t{}".format(i, res))

print(optimizer.max)

W0226 13:05:54.301614 19620 deprecation_wrapper.py:119] From E:\ProgramData\Anaconda3\envs\tf_gpu\lib\site-packages\keras\backend\tensorflow_backend.py:74: The name tf.get_default_graph is deprecated. Please use tf.compat.v1.get_default_graph instead.

W0226 13:05:54.303613 19620 deprecation_wrapper.py:119] From E:\ProgramData\Anaconda3\envs\tf_gpu\lib\site-packages\keras\backend\tensorflow_backend.py:517: The name tf.placeholder is deprecated. Please use tf.compat.v1.placeholder instead.

W0226 13:05:54.306617 19620 deprecation_wrapper.py:119] From E:\ProgramData\Anaconda3\envs\tf_gpu\lib\site-packages\keras\backend\tensorflow_backend.py:4138: The name tf.random_uniform is deprecated. Please use tf.random.uniform instead.

W0226 13:05:54.324613 19620 deprecation_wrapper.py:119] From E:\ProgramData\Anaconda3\envs\tf_gpu\lib\site-packages\keras\backend\tensorflow_backend.py:133: The name tf.placeholder_with_default is deprecated. Please use tf.compat.v1.placeholder_with_default instead.

|   iter    |  target   | dropou... | dropou... | dropou... | dropou... | learni... |
-------------------------------------------------------------------------------------


W0226 13:05:54.498610 19620 deprecation_wrapper.py:119] From E:\ProgramData\Anaconda3\envs\tf_gpu\lib\site-packages\keras\optimizers.py:790: The name tf.train.Optimizer is deprecated. Please use tf.compat.v1.train.Optimizer instead.

W0226 13:05:54.505618 19620 deprecation_wrapper.py:119] From E:\ProgramData\Anaconda3\envs\tf_gpu\lib\site-packages\keras\backend\tensorflow_backend.py:3376: The name tf.log is deprecated. Please use tf.math.log instead.

W0226 13:05:54.513616 19620 deprecation.py:323] From E:\ProgramData\Anaconda3\envs\tf_gpu\lib\site-packages\tensorflow\python\ops\nn_impl.py:180: add_dispatch_support.<locals>.wrapper (from tensorflow.python.ops.array_ops) is deprecated and will be removed in a future version.
Instructions for updating:
Use tf.where in 2.0, which has the same broadcast rule as np.where


Train on 785336 samples, validate on 261779 samples
Epoch 1/100
Epoch 2/100
Epoch 3/100
Epoch 4/100
Epoch 5/100
Epoch 6/100
Epoch 7/100
Epoch 8/100
Epoch 9/100
Epoch 10/100
Epoch 11/100
Epoch 12/100
Epoch 13/100
Epoch 14/100
Epoch 15/100
Epoch 16/100
Epoch 17/100
Epoch 18/100
Epoch 19/100
Epoch 20/100
Epoch 21/100
Epoch 22/100
Epoch 23/100
Epoch 24/100
Epoch 25/100
Epoch 26/100
Epoch 27/100
Epoch 28/100
Epoch 29/100
Epoch 30/100
Epoch 31/100
Epoch 32/100
Epoch 33/100
Epoch 34/100
Epoch 35/100
Epoch 36/100
Epoch 37/100
Epoch 38/100
Epoch 39/100
Epoch 40/100
Epoch 41/100
Epoch 42/100
Epoch 43/100
Epoch 44/100
Epoch 45/100
Epoch 46/100
Epoch 47/100
Epoch 48/100
Epoch 49/100
Epoch 50/100
Epoch 51/100
Epoch 52/100
Epoch 53/100
Epoch 54/100
Epoch 55/100
Epoch 56/100
Epoch 57/100
Epoch 58/100
Epoch 59/100
Epoch 60/100
Epoch 61/100
Epoch 62/100
Epoch 63/100
Epoch 64/100
Epoch 65/100
Epoch 66/100
Epoch 67/100
Epoch 68/100
Epoch 69/100
Epoch 70/100
Epoch 71/100
Epoch 72/100
Epoch 73/100
Epoch 74

Epoch 18/100
Epoch 19/100
Epoch 20/100
Epoch 21/100
Epoch 22/100
Epoch 23/100
Epoch 24/100
Epoch 25/100
Epoch 26/100
Epoch 27/100
Epoch 28/100
Epoch 29/100
Epoch 30/100
Epoch 31/100
Epoch 32/100
Epoch 33/100
Epoch 34/100
Epoch 35/100
Epoch 36/100
Epoch 37/100
Epoch 38/100
Epoch 39/100
Epoch 40/100
Epoch 41/100
Epoch 42/100
Epoch 43/100
Epoch 44/100
Epoch 45/100
Epoch 46/100
Epoch 47/100
Epoch 48/100
Epoch 49/100
Epoch 50/100
Epoch 51/100
Epoch 52/100
Epoch 53/100
Epoch 54/100
Epoch 55/100
Epoch 56/100
Epoch 57/100
Epoch 58/100
Epoch 59/100
Epoch 60/100
Epoch 61/100
Epoch 62/100
Epoch 63/100
Epoch 64/100
Epoch 65/100
Epoch 66/100
Epoch 67/100
Epoch 68/100
Epoch 69/100
Epoch 70/100
Epoch 71/100
Epoch 72/100
Epoch 73/100
Epoch 74/100
Epoch 75/100
Epoch 76/100
Epoch 77/100
Epoch 78/100
Epoch 79/100
Epoch 80/100
Epoch 81/100
Epoch 82/100
Epoch 83/100
Epoch 84/100
Epoch 85/100
Epoch 86/100
Epoch 87/100
Epoch 88/100
Epoch 89/100
Epoch 90/100
Epoch 91/100
Epoch 92/100
Epoch 93/100
Epoch 94/100

Epoch 39/100
Epoch 40/100
Epoch 41/100
Epoch 42/100
Epoch 43/100
Epoch 44/100
Epoch 45/100
Epoch 46/100
Epoch 47/100
Epoch 48/100
Epoch 49/100
Epoch 50/100
Epoch 51/100
Epoch 52/100
Epoch 53/100
Epoch 54/100
Epoch 55/100
Epoch 56/100
Epoch 57/100
Epoch 58/100
Epoch 59/100
Epoch 60/100
Epoch 61/100
Epoch 62/100
Epoch 63/100
Epoch 64/100
Epoch 65/100
Epoch 66/100
Epoch 67/100
Epoch 68/100
Epoch 69/100
Epoch 70/100
Epoch 71/100
Epoch 72/100
Epoch 73/100
Epoch 74/100
Epoch 75/100
Epoch 76/100
Epoch 77/100
Epoch 78/100
Epoch 79/100
Epoch 80/100
Epoch 81/100
Epoch 82/100
Epoch 83/100
Epoch 84/100
Epoch 85/100
Epoch 86/100
Epoch 87/100
Epoch 88/100
Epoch 89/100
Epoch 90/100
Epoch 91/100
Epoch 92/100
Epoch 93/100
Epoch 94/100
Epoch 95/100
Epoch 96/100
Epoch 97/100
Epoch 98/100
Epoch 99/100
Epoch 100/100
DNN model trained in 14048.876 seconds
val loss: 10.160949827277443
val accuracy: 0.3626455903640498
|  11       |  0.3626   |  0.1152   |  0.1893   |  0.1671   |  0.1929   |  0.004271 |
Train 

Epoch 60/100
Epoch 61/100
Epoch 62/100
Epoch 63/100
Epoch 64/100
Epoch 65/100
Epoch 66/100
Epoch 67/100
Epoch 68/100
Epoch 69/100
Epoch 70/100
Epoch 71/100
Epoch 72/100
Epoch 73/100
Epoch 74/100
Epoch 75/100
Epoch 76/100
Epoch 77/100
Epoch 78/100
Epoch 79/100
Epoch 80/100
Epoch 81/100
Epoch 82/100
Epoch 83/100
Epoch 84/100
Epoch 85/100
Epoch 86/100
Epoch 87/100
Epoch 88/100
Epoch 89/100
Epoch 90/100
Epoch 91/100
Epoch 92/100
Epoch 93/100
Epoch 94/100
Epoch 95/100
Epoch 96/100
Epoch 97/100
Epoch 98/100
Epoch 99/100
Epoch 100/100
DNN model trained in 14275.877 seconds
val loss: 10.160949827277443
val accuracy: 0.3626455903640498
|  16       |  0.3626   |  0.293    |  0.2924   |  0.1423   |  0.1087   |  0.008607 |
Train on 785336 samples, validate on 261779 samples
Epoch 1/100
Epoch 2/100
Epoch 3/100
Epoch 4/100
Epoch 5/100
Epoch 6/100
Epoch 7/100
Epoch 8/100
Epoch 9/100
Epoch 10/100
Epoch 11/100
Epoch 12/100
Epoch 13/100
Epoch 14/100
Epoch 15/100
Epoch 16/100
Epoch 17/100
Epoch 18/100
Ep